<a href="https://colab.research.google.com/github/Akshat-kumar-jain/Hamoye_Internship/blob/main/Hamoye_Stage_B_of_Internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hamoye - Stage B
# Machine Learning: Regression - Predicting Energy Efficiency of Buildings
# Submitted by Akshat Kumar Jain ( iakshatkumarjain@gmail.com )

#Dataset Description

`The dataset for the remainder of this quiz is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.`


## Attribute Information:

Date, time year-month-day hour:minute:second

Appliances, energy use in Wh

lights, energy use of light fixtures in the house in Wh

T1, Temperature in kitchen area, in Celsius

RH_1, Humidity in kitchen area, in %

T2, Temperature in living room area, in Celsius

RH_2, Humidity in living room area, in %

T3, Temperature in laundry room area

RH_3, Humidity in laundry room area, in %

T4, Temperature in office room, in Celsius

RH_4, Humidity in office room, in %

T5, Temperature in bathroom, in Celsius

RH_5, Humidity in bathroom, in %

T6, Temperature outside the building (north side), in Celsius

RH_6, Humidity outside the building (north side), in %

T7, Temperature in ironing room , in Celsius

RH_7, Humidity in ironing room, in %

T8, Temperature in teenager room 2, in Celsius

RH_8, Humidity in teenager room 2, in %

T9, Temperature in parents room, in Celsius

RH_9, Humidity in parents room, in %

To, Temperature outside (from Chievres weather station), in Celsius

Pressure (from Chievres weather station), in mm Hg

RH_out, Humidity outside (from Chievres weather station), in %

Wind speed (from Chievres weather station), in m/s

Visibility (from Chievres weather station), in km

Tdewpoint (from Chievres weather station), Â °C

rv1, Random variable 1, nondimensional

rv2, Random variable 2, nondimensional





#---------
To answer some questions, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set.

# Importing The Needed Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Reading in The dataset

In [ ]:
data = pd.read_csv("energydata_complete.csv")
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


# Inspecting The Dataset

In [ ]:
data.shape

(19735, 29)

In [ ]:
data.size

572315

In [ ]:
data.dtypes

date            object
Appliances       int64
lights           int64
T1             float64
RH_1           float64
T2             float64
RH_2           float64
T3             float64
RH_3           float64
T4             float64
RH_4           float64
T5             float64
RH_5           float64
T6             float64
RH_6           float64
T7             float64
RH_7           float64
T8             float64
RH_8           float64
T9             float64
RH_9           float64
T_out          float64
Press_mm_hg    float64
RH_out         float64
Windspeed      float64
Visibility     float64
Tdewpoint      float64
rv1            float64
rv2            float64
dtype: object

In [ ]:
data.dtypes.value_counts()

float64    26
int64       2
object      1
dtype: int64

In [ ]:
data.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [ ]:
data.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [ ]:
data.duplicated().sum()

0

# Checking The Correlation of The dataset

In [ ]:
data.corr()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
Appliances,1.000000,0.197278,0.055447,0.086031,0.120073,-0.060465,0.085060,0.036292,0.040281,0.016965,...,0.010010,-0.051462,0.099155,-0.034885,-0.152282,0.087122,0.000230,0.015353,-0.011145,-0.011145
lights,0.197278,1.000000,-0.023528,0.106968,-0.005622,0.050985,-0.097393,0.131161,-0.008859,0.114936,...,-0.157592,-0.008766,-0.074424,-0.010576,0.068543,0.060281,0.020038,-0.036322,0.000521,0.000521
T1,0.055447,-0.023528,1.000000,0.164006,0.836834,-0.002509,0.892402,-0.028550,0.877001,0.097861,...,0.844777,0.071756,0.682846,-0.150574,-0.345481,-0.087654,-0.076210,0.571309,-0.006203,-0.006203
RH_1,0.086031,0.106968,0.164006,1.000000,0.269839,0.797535,0.253230,0.844677,0.106180,0.880359,...,0.115263,0.764001,0.340767,-0.293957,0.274126,0.204932,-0.021057,0.639106,-0.000699,-0.000699
T2,0.120073,-0.005622,0.836834,0.269839,1.000000,-0.165610,0.735245,0.121497,0.762066,0.231563,...,0.675535,0.157346,0.792255,-0.133028,-0.505291,0.052495,-0.069721,0.582602,-0.011087,-0.011087
RH_2,-0.060465,0.050985,-0.002509,0.797535,-0.165610,1.000000,0.137319,0.678326,-0.047304,0.721435,...,0.054544,0.676467,0.033674,-0.255646,0.584911,0.069190,-0.005368,0.499152,0.006275,0.006275
T3,0.085060,-0.097393,0.892402,0.253230,0.735245,0.137319,1.000000,-0.011234,0.852778,0.122737,...,0.901324,0.134602,0.699417,-0.189974,-0.281718,-0.100776,-0.102310,0.645886,-0.005194,-0.005194
RH_3,0.036292,0.131161,-0.028550,0.844677,0.121497,0.678326,-0.011234,1.000000,-0.140457,0.898978,...,-0.195270,0.833538,0.118207,-0.233274,0.356192,0.263188,0.017041,0.414387,-0.000477,-0.000477
T4,0.040281,-0.008859,0.877001,0.106180,0.762066,-0.047304,0.852778,-0.140457,1.000000,-0.048650,...,0.889439,-0.025549,0.663478,-0.075292,-0.388602,-0.185747,-0.104768,0.519471,-0.001815,-0.001815
RH_4,0.016965,0.114936,0.097861,0.880359,0.231563,0.721435,0.122737,0.898978,-0.048650,1.000000,...,-0.044518,0.856591,0.293289,-0.250748,0.336813,0.300192,0.002636,0.616509,-0.001787,-0.001787


## Checking that the values in the columns 'rv1' and 'rv2' are the same

In [ ]:
data[["rv1", "rv2"]]

,rv1,rv2
0,13.275433,13.275433
1,18.606195,18.606195
2,28.642668,28.642668
3,45.410389,45.410389
4,10.084097,10.084097
...,...,...
19730,43.096812,43.096812
19731,49.282940,49.282940
19732,29.199117,29.199117
19733,6.322784,6.322784


In [ ]:
assert all(data["rv1"]) == all(data["rv2"])

# Inspecting each column of the dataset for possible patterns

In [ ]:
data["Appliances"].sort_values().unique()

array([  10,   20,   30,   40,   50,   60,   70,   80,   90,  100,  110,
        120,  130,  140,  150,  160,  170,  180,  190,  200,  210,  220,
        230,  240,  250,  260,  270,  280,  290,  300,  310,  320,  330,
        340,  350,  360,  370,  380,  390,  400,  410,  420,  430,  440,
        450,  460,  470,  480,  490,  500,  510,  520,  530,  540,  550,
        560,  570,  580,  590,  600,  610,  620,  630,  640,  650,  660,
        670,  680,  690,  700,  710,  720,  730,  740,  750,  760,  770,
        780,  790,  800,  820,  830,  840,  850,  860,  870,  880,  890,
        900,  910, 1070, 1080], dtype=int64)

In [ ]:
data["Appliances"].value_counts().sort_index()

10         9
20       343
30       723
40      2019
50      4368
        ... 
890        1
900        1
910        1
1070       1
1080       1
Name: Appliances, Length: 92, dtype: int64

In [ ]:
data["lights"].unique()

array([30, 40, 50, 70, 60, 10, 20,  0], dtype=int64)

In [ ]:
data["lights"].sort_values().value_counts()

0     15252
10     2212
20     1624
30      559
40       77
50        9
60        1
70        1
Name: lights, dtype: int64

In [ ]:
len(data["T1"].unique())

722

In [ ]:
data["T1"].value_counts().sort_index()

16.790000    10
16.823333     2
16.856667     2
16.890000    19
16.963333     1
             ..
26.066667     3
26.100000    15
26.166667     3
26.200000     7
26.260000     3
Name: T1, Length: 722, dtype: int64

In [ ]:
len(data["RH_1"].unique())

2547

In [ ]:
data["RH_1"].value_counts().sort_index()

27.023333    1
27.233333    1
27.360000    1
27.430000    1
27.545000    1
            ..
57.423333    1
57.496667    1
57.663333    1
59.633333    1
63.360000    1
Name: RH_1, Length: 2547, dtype: int64

In [ ]:
data["T2"].unique()

array([19.2       , 19.23      , 19.35666667, ..., 25.978     ,
       25.754     , 25.62857143])

In [ ]:
data["T2"].value_counts()

19.200000    331
19.600000    325
19.500000    318
19.700000    313
19.290000    301
            ... 
20.514286      1
21.772000      1
19.125000      1
24.518018      1
28.238571      1
Name: T2, Length: 1650, dtype: int64

In [ ]:
data["RH_2"].unique()

array([44.79      , 44.7225    , 44.62666667, ..., 42.02571429,
       42.08      , 42.76857143])

In [ ]:
data["RH_2"].value_counts()

40.000000    135
41.500000    124
43.700000    110
41.290000    109
43.900000    108
            ... 
33.456667      1
39.311667      1
35.640000      1
36.975000      1
37.444444      1
Name: RH_2, Length: 3376, dtype: int64

In [ ]:
data["T3"].unique()

array([19.79      , 19.73      , 19.89      , ..., 28.56666667,
       28.35666667, 27.05      ])

In [ ]:
data["T3"].value_counts()

22.390000    433
22.100000    372
21.790000    369
22.600000    338
20.200000    329
            ... 
27.197500      1
28.335714      1
24.896104      1
19.960000      1
22.238571      1
Name: T3, Length: 1426, dtype: int64

In [ ]:
data["RH_3"].unique()

array([44.73      , 44.79      , 44.93333333, ..., 42.99333333,
       41.79666667, 41.69      ])

In [ ]:
data["RH_3"].value_counts()

37.590000    243
37.500000    205
38.290000    199
37.200000    197
37.900000    188
            ... 
41.123333      1
36.595000      1
42.487500      1
43.356000      1
42.993333      1
Name: RH_3, Length: 2618, dtype: int64

In [ ]:
data["T4"].unique()

array([19.        , 18.92666667, 18.89      , ..., 23.41337662,
       23.40168831, 24.53333333])

In [ ]:
data["T4"].value_counts()

20.390000    348
20.290000    345
20.600000    337
20.100000    335
19.790000    333
            ... 
24.750000      1
23.360000      1
21.075000      1
23.140000      1
24.056234      1
Name: T4, Length: 1390, dtype: int64

In [ ]:
data["RH_4"].unique()

array([45.56666667, 45.9925    , 45.89      , ..., 43.45      ,
       44.7225    , 45.33333333])

In [ ]:
data["RH_4"].value_counts()

38.590000    169
38.200000    166
34.790000    146
38.090000    145
38.290000    140
            ... 
47.366126      1
43.356000      1
48.506937      1
41.523636      1
36.022500      1
Name: RH_4, Length: 2987, dtype: int64

In [ ]:
data["T5"].unique()

array([17.16666667, 17.2       , 17.13333333, ..., 22.76      ,
       22.63333333, 22.945     ])

In [ ]:
data["T5"].value_counts()

18.600000    385
18.700000    373
19.290000    370
20.000000    361
18.500000    357
            ... 
20.514286      1
20.903750      1
21.275000      1
22.684234      1
16.105556      1
Name: T5, Length: 2263, dtype: int64

In [ ]:
data["RH_5"].unique()

array([55.2  , 55.09 , 55.03 , ..., 54.43 , 53.25 , 52.645])

In [ ]:
data["RH_5"].value_counts()

49.000000    89
45.000000    81
46.700000    78
47.500000    76
45.900000    75
             ..
76.150000     1
38.616000     1
50.117500     1
55.450000     1
61.396667     1
Name: RH_5, Length: 7571, dtype: int64

In [ ]:
data["T6"].unique()

array([ 7.02666667,  6.83333333,  6.56      , ..., 23.62666667,
       22.43333333, 21.02666667])

In [ ]:
data["T6"].value_counts()

 6.400000     86
 5.590000     84
 9.100000     78
 9.600000     78
 6.300000     76
              ..
 12.812857     1
 15.304286     1
-5.350000      1
 13.838571     1
 7.745000      1
Name: T6, Length: 4446, dtype: int64

In [ ]:
data["RH_6"].unique()

array([84.25666667, 84.06333333, 83.15666667, ...,  6.56666667,
        3.55666667,  1.69666667])

In [ ]:
data["RH_6"].value_counts()

1.000000     1452
99.900000     751
92.300000      53
92.190000      38
87.690000      29
             ... 
56.550000       1
33.666667       1
90.143333       1
22.900000       1
7.000000        1
Name: RH_6, Length: 9709, dtype: int64

In [ ]:
data["T7"].unique()

array([17.2       , 17.13333333, 17.1       , ..., 23.90571429,
       23.96857143, 24.31      ])

In [ ]:
data["T7"].value_counts()

18.500000    453
19.200000    450
19.290000    345
19.000000    302
18.390000    297
            ... 
20.122222      1
17.275000      1
20.127778      1
15.594444      1
17.867778      1
Name: T7, Length: 1955, dtype: int64

In [ ]:
data["RH_7"].unique()

array([41.62666667, 41.56      , 41.43333333, ..., 44.76428571,
       44.41428571, 44.29571429])

In [ ]:
data["RH_7"].value_counts()

33.900000    131
33.590000    118
33.700000    104
30.200000     90
36.290000     86
            ... 
33.797143      1
40.526667      1
37.952941      1
33.081111      1
46.035714      1
Name: RH_7, Length: 5891, dtype: int64

In [ ]:
data["T8"].unique()

array([18.2       , 18.1       , 18.06666667, ..., 23.71103896,
       24.625     , 24.6625    ])

In [ ]:
data["T8"].value_counts()

22.790000    386
21.790000    301
22.700000    289
23.390000    276
21.390000    275
            ... 
24.478000      1
24.461688      1
24.120000      1
16.957222      1
24.272364      1
Name: T8, Length: 2228, dtype: int64

In [ ]:
data["RH_8"].unique()

array([48.9       , 48.86333333, 48.73      , ..., 50.445     ,
       50.074     , 49.51875   ])

In [ ]:
data["RH_8"].value_counts()

39.500000    95
39.290000    80
39.590000    67
39.200000    65
43.700000    63
             ..
41.720000     1
49.027222     1
43.390000     1
39.761429     1
48.756111     1
Name: RH_8, Length: 6649, dtype: int64

In [ ]:
data["T9"].unique()

array([17.03333333, 17.06666667, 17.        , 16.89      , 16.96333333,
       16.82333333, 17.1       , 17.13333333, 17.2       , 17.15      ,
       17.05      , 17.16666667, 16.92666667, 16.79      , 16.85666667,
       16.865     , 16.73      , 16.7       , 16.66666667, 16.65      ,
       16.6       , 16.63333333, 16.745     , 16.76      , 17.26      ,
       16.98166667, 17.23      , 17.29      , 17.32333333, 17.39      ,
       17.42666667, 17.46333333, 17.5       , 17.56666667, 17.6       ,
       17.66666667, 17.65      , 17.7       , 17.63333333, 17.34      ,
       17.35666667, 17.245     , 17.53333333, 17.73      , 17.79      ,
       17.865     , 17.89      , 17.96333333, 17.92666667, 17.82333333,
       17.85666667, 18.        , 18.03333333, 18.1       , 18.13333333,
       18.2       , 18.16666667, 17.76      , 17.745     , 18.06666667,
       18.2225    , 18.29      , 18.39      , 18.5       , 18.22333333,
       16.84      , 16.5       , 16.53333333, 16.675     , 16.56

In [ ]:
data["T9"].value_counts()

20.500000    487
18.200000    476
20.290000    449
19.600000    443
18.000000    422
            ... 
23.129364      1
23.037143      1
17.478000      1
23.405714      1
22.238571      1
Name: T9, Length: 924, dtype: int64

In [ ]:
data["RH_9"].unique()

array([45.53  , 45.56  , 45.5   , ..., 46.718 , 46.8175, 46.845 ])

In [ ]:
data["RH_9"].value_counts()

39.200000    165
39.790000    159
39.700000    148
38.500000    136
37.700000    127
            ... 
38.734000      1
42.514000      1
52.163333      1
34.666667      1
39.600000      1
Name: RH_9, Length: 3388, dtype: int64

In [ ]:
data["T_out"].unique()

array([ 6.6       ,  6.48333333,  6.36666667, ..., 20.91666667,
       21.55      , 22.13333333])

In [ ]:
data["T_out"].value_counts()

9.800000     119
9.600000     107
5.000000     105
10.100000    105
5.500000      96
            ... 
4.257143       1
4.492857       1
23.133333      1
3.828571       1
21.333333      1
Name: T_out, Length: 1730, dtype: int64

In [ ]:
data["RH_out"].unique()

array([ 92.        ,  91.83333333,  91.66666667,  91.5       ,
        91.33333333,  91.16666667,  91.        ,  90.5       ,
        90.        ,  89.5       ,  89.        ,  88.5       ,
        88.        ,  87.83333333,  87.66666667,  87.5       ,
        87.33333333,  87.16666667,  87.        ,  88.66666667,
        90.33333333,  92.16666667,  92.33333333,  92.5       ,
        92.66666667,  92.83333333,  93.        ,  93.66666667,
        94.33333333,  95.        ,  94.66666667,  94.        ,
        93.33333333,  90.16666667,  89.33333333,  86.83333333,
        86.        ,  85.16666667,  84.33333333,  83.5       ,
        82.66666667,  81.83333333,  81.        ,  80.66666667,
        80.33333333,  80.        ,  79.66666667,  79.33333333,
        79.        ,  80.16666667,  78.5       ,  77.66666667,
        76.83333333,  76.        ,  77.        ,  78.        ,
        82.        ,  80.83333333,  77.33333333,  76.16666667,
        75.        ,  76.66666667,  78.33333333,  81.66

In [ ]:
data["RH_out"].value_counts()

95.000000    470
92.000000    342
88.000000    341
91.000000    332
93.000000    318
            ... 
59.777778      1
80.444444      1
92.571429      1
91.750000      1
93.583333      1
Name: RH_out, Length: 566, dtype: int64

In [ ]:
data["Press_mm_hg"].unique()

array([733.5  , 733.6  , 733.7  , ..., 757.175, 757.225, 757.275])

In [ ]:
data["Press_mm_hg"].value_counts()

757.300000    142
756.200000    110
764.500000     98
756.600000     97
755.800000     95
             ... 
743.216667      1
740.783333      1
731.866667      1
733.833333      1
735.083333      1
Name: Press_mm_hg, Length: 2189, dtype: int64

In [ ]:
data["Windspeed"].unique()

array([ 7.        ,  6.66666667,  6.33333333,  6.        ,  5.66666667,
        5.33333333,  5.        ,  5.16666667,  5.5       ,  5.83333333,
        6.16666667,  6.5       ,  6.83333333,  4.83333333,  4.66666667,
        4.5       ,  4.33333333,  4.16666667,  4.        ,  7.16666667,
        7.33333333,  7.5       ,  7.66666667,  7.83333333,  8.        ,
        3.83333333,  3.66666667,  3.5       ,  3.33333333,  3.16666667,
        3.        ,  8.33333333,  9.        ,  8.83333333,  8.66666667,
        8.5       ,  8.16666667,  9.33333333,  9.66666667, 10.        ,
        9.16666667,  9.5       ,  9.83333333, 10.16666667, 10.33333333,
       10.5       , 10.66666667, 10.83333333, 11.        ,  2.5       ,
        2.        ,  2.16666667,  2.33333333,  2.66666667,  2.83333333,
        1.66666667,  1.33333333,  1.        ,  1.83333333,  1.5       ,
        1.16666667,  0.83333333,  0.66666667,  0.5       ,  0.33333333,
        0.16666667,  0.        ,  1.94444444,  1.88888889,  1.77

In [ ]:
data["Windspeed"].value_counts()

1.000000    2133
2.000000    2007
3.000000    1707
4.000000    1594
5.000000    1084
            ... 
5.714286       1
8.222222       1
4.888889       1
1.111111       1
5.285714       1
Name: Windspeed, Length: 189, dtype: int64

In [ ]:
data["Visibility"].unique()

array([63.        , 59.16666667, 55.33333333, 51.5       , 47.66666667,
       43.83333333, 40.        , 37.        , 34.        , 31.        ,
       28.        , 25.        , 22.        , 38.16666667, 36.33333333,
       34.5       , 32.66666667, 30.83333333, 29.        , 38.        ,
       36.        , 32.        , 30.        , 37.83333333, 35.66666667,
       33.5       , 31.33333333, 29.16666667, 27.        , 26.        ,
       24.        , 23.        , 21.        , 21.5       , 22.5       ,
       23.5       , 23.66666667, 23.33333333, 22.66666667, 22.33333333,
       22.16666667, 22.83333333, 25.83333333, 28.66666667, 31.5       ,
       34.33333333, 37.16666667, 36.66666667, 33.33333333, 26.66666667,
       20.        , 24.5       , 27.5       , 28.33333333, 27.66666667,
       27.33333333, 27.16666667, 27.83333333, 28.16666667, 28.5       ,
       28.83333333, 25.33333333, 21.33333333, 32.5       , 35.        ,
       35.83333333, 37.5       , 38.33333333, 39.16666667, 42.33

In [ ]:
data["Visibility"].value_counts()

40.000000    7224
29.000000     449
36.333333     257
23.000000     247
28.000000     245
             ... 
33.277778       1
35.722222       1
61.916667       1
2.500000        1
32.361111       1
Name: Visibility, Length: 413, dtype: int64

In [ ]:
data["Tdewpoint"].unique()

array([ 5.3       ,  5.2       ,  5.1       , ...,  9.925     ,
       10.24166667, 11.51666667])

In [ ]:
data["Tdewpoint"].value_counts()

0.000000     123
1.400000     110
2.100000     108
2.900000      98
2.000000      95
            ... 
3.321429       1
11.116667      1
2.330952       1
1.608333       1
5.677778       1
Name: Tdewpoint, Length: 1409, dtype: int64

# Creating a new column name in a list

In [1]:
new_name = ["Appliances", "temp_in_kitchen", "RH_in_kitchen", "temp_in_living_room", "RH_in_living_room", "temp_in_laundry_room", "RH_in_laundry_room", "temp_in_office_room", "RH_in_office_room", "temp_in_bathroom", "RH_in_bathroom", "temp_outside_building_northside", "RH_outside_building_northside", "temp_in_ironing_room", "RH_in_ironing_room", "temp_in_teenager_room2", "RH_in_teenager_room2", "temp_in_parents_room", "RH_in_parents_room2", "temp_weather_station", "pressure_waether_station", "RH_weather_station", "windspeed_weather_station", "visibility_weather_station", "Tdewpoint_weather_station", "random_variable1", "random_variable2"]

# Normalizing the dataset and dropping out the unwanted columns

In [ ]:
scaler = MinMaxScaler()
new_data = pd.DataFrame(scaler.fit_transform(data.drop(["date", "lights"], axis = 1)), columns = new_name)
new_data.head()

,Appliances,temp_in_kitchen,RH_in_kitchen,temp_in_living_room,RH_in_living_room,temp_in_laundry_room,RH_in_laundry_room,temp_in_office_room,RH_in_office_room,temp_in_bathroom,...,temp_in_parents_room,RH_in_parents_room2,temp_weather_station,pressure_waether_station,RH_weather_station,windspeed_weather_station,visibility_weather_station,Tdewpoint_weather_station,random_variable1,random_variable2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.037383,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.037383,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.046729,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [ ]:
new_data.dtypes.value_counts()

float64    27
dtype: int64

# Setting the Descriptor Dataset

In [ ]:
X = new_data.drop(["Appliances"], axis = 1)
X

,temp_in_kitchen,RH_in_kitchen,temp_in_living_room,RH_in_living_room,temp_in_laundry_room,RH_in_laundry_room,temp_in_office_room,RH_in_office_room,temp_in_bathroom,RH_in_bathroom,...,temp_in_parents_room,RH_in_parents_room2,temp_weather_station,pressure_waether_station,RH_weather_station,windspeed_weather_station,visibility_weather_station,Tdewpoint_weather_station,random_variable1,random_variable2
0,0.327350,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.327350,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.327350,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.327350,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.327350,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,0.926786,0.537657,0.711655,0.606309,0.830841,0.579374,0.864865,0.765258,0.752031,0.339590,...,0.864724,0.729443,0.891747,0.602326,0.416667,0.238095,0.348718,0.901961,0.861981,0.861981
19731,0.919747,0.536006,0.701769,0.607836,0.825302,0.582178,0.864865,0.765258,0.754897,0.338487,...,0.864724,0.729443,0.887460,0.602326,0.421053,0.250000,0.361538,0.900452,0.985726,0.985726
19732,0.919747,0.538666,0.692651,0.627198,0.818378,0.603988,0.864865,0.771233,0.754897,0.337585,...,0.864724,0.729443,0.883173,0.602326,0.425439,0.261905,0.374359,0.898944,0.583979,0.583979
19733,0.919747,0.549491,0.677054,0.634717,0.805085,0.585294,0.864865,0.773794,0.752031,0.336583,...,0.864724,0.730581,0.878885,0.602326,0.429825,0.273810,0.387179,0.897436,0.126371,0.126371


In [ ]:
print(new_data.shape)
print(X.shape)

(19735, 27)
(19735, 26)


In [ ]:
for col in new_data.columns :
    if col not in X.columns :
        print(col)
    else :
        pass

Appliances


# Setting The Target Variable

In [ ]:
y = np.array(new_data["Appliances"])
y.shape

(19735,)

# Splitting The Dataset into 'Train Test Split'

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Checking The shape of the Train and Test sets

In [ ]:
print(f"x_train:{x_train.shape}\ny_train{y_train.shape}\nx_test:{x_test.shape}\ny_test{y_test.shape}")

x_train:(13814, 26)
y_train(13814,)
x_test:(5921, 26)
y_test(5921,)


In [ ]:
linear_model = LinearRegression()

In [ ]:
linear_model.fit(x_train, y_train)

LinearRegression()

In [ ]:
linear_predict = linear_model.predict(x_test)

## Question 1

In [ ]:
print("The Answer is Guassian prior")

The Answer is Guassian prior


## Question 2

In [ ]:
print("Cross Validation is guaranteed to prevent overfitting")

Cross Validation is guaranteed to prevent overfitting


## Question 3

In [ ]:
print("Ridge Regression oftens sets several of the weights to zero")

Ridge Regression oftens sets several of the weights to zero


## Question 4

In [ ]:
print("In the different terms of the bias-variance tradeoff, which of the following is substantially more harmful to the test error than the training error?")
print("\nVariance")

In the different terms of the bias-variance tradeoff, which of the following is substantially more harmful to the test error than the training error?

Variance


## Question 5

In [ ]:
print("What can you use to find the best fit line for data in Linear Regression ?")
print("\nNone of The Above")

What can you use to find the best fit line for data in Linear Regression ?

None of The Above


## Question 6

In [ ]:
print("Which of the following sentence is true about outliers in Linear Regression:")
print("\nA and B above")

Which of the following sentence is true about outliers in Linear Regression:

A and B above


## Question 7

In [ ]:
print("How many coefficients do you need to estimate a simple linear regression model (One independent variable) ?")
print("\n1")

How many coefficients do you need to estimate a simple linear regression model (One independent variable) ?

1


## Question 8

In [ ]:
print("Adding more basis functions in a linear model: ")
print("\nDecrease the Model Bias")

Adding more basis functions in a linear model: 

Decrease the Model Bias


## Question 9

In [ ]:
print("A best fit line relating X and Y has a R-Squared value of 0.75. How do I interpret this information ?")
print("\nX and Y are 75% related")

A best fit line relating X and Y has a R-Squared value of 0.75. How do I interpret this information ?

X and Y are 75% related


## Question 10

In [ ]:
print("The Lasso can be interpreted as least-squares linear regression where: ")
print("\nWeights are regularized with the L1 norm")

The Lasso can be interpreted as least-squares linear regression where: 

Weights are regularized with the L1 norm


## Question 11

In [ ]:
print("Which of these is not an assumption of Linear Regression ?")
print("\nMultivariate Normality")

Which of these is not an assumption of Linear Regression ?

Multivariate Normality


## Question 12 : From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [ ]:
qi_x  = np.array(x_train["temp_in_living_room"]).reshape(-1, 1)
qi_x.ndim

2

In [ ]:
qi_y = np.array(x_train["temp_outside_building_northside"])
qi_y.ndim

1

In [ ]:
qi_linear_model = LinearRegression()

In [ ]:
qi_model = qi_linear_model.fit(qi_x, qi_y)

In [ ]:
qi_predict = qi_linear_model.predict(np.array(x_test["temp_in_living_room"]).reshape(-1, 1))

In [ ]:
qi_r2 = r2_score(qi_predict, np.array(x_test["temp_outside_building_northside"]))
round(qi_r2, 2)


## Question 13 :What is the Mean Absolute Error (in two decimal places)?

In [ ]:
qi_mae = mean_absolute_error(np.array(x_test["temp_outside_building_northside"]), qi_predict)
round(qi_mae, 2)

0.08

## Question 14 : What is the Residual Sum of Squares (in two decimal places)?

In [ ]:
qi_rss = np.sum(np.square(np.array(x_test["temp_outside_building_northside"]) - qi_predict))
round(qi_rss, 2)

66.12

## Question 15 : What is the Root Mean Squared Error (in three decimal places)?

In [ ]:
qi_rmse = np.sqrt(mean_squared_error(np.array(x_test["temp_outside_building_northside"]), qi_predict))
round(qi_rmse, 3) 

0.106

## Question 16 : What is the Coefficient of Determination (in two decimal places)?

In [ ]:
# Coefficient of Determination is the same as r2_score
qi_cod = r2_score(qi_predict, np.array(x_test["temp_outside_building_northside"]))
round(qi_cod, 2)

0.44

## Question 17 : Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?







In [ ]:
#comparing the effects of Regularisation

def get_weights_df(model, feat, col_name) :
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features", col_name]
    weights_df[col_name].round(3)
    
    return weights_df

In [ ]:
len(linear_model.coef_)

26

In [ ]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
linear_model_weights.nsmallest(1, "Linear_Model_Weight")

,Features,Linear_Model_Weight
0,RH_in_living_room,-0.456698


In [ ]:
linear_model_weights.nlargest(1, "Linear_Model_Weight")

,Features,Linear_Model_Weight
25,RH_in_kitchen,0.553547


## Question 18 : Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [ ]:
ridge_model = Ridge(alpha=0.4)

In [ ]:
ridge_model.fit(qi_x, qi_y)

Ridge(alpha=0.4)

In [ ]:
qi_ridge_predict = ridge_model.predict(np.array(x_test["temp_in_living_room"]).reshape(-1, 1))

In [ ]:
qi_ridge_rmse = np.sqrt(mean_squared_error(np.array(x_test["temp_outside_building_northside"]), qi_ridge_predict))
round(qi_ridge_rmse, 3) 

0.106

In [ ]:
assert round(qi_rmse, 3) == round(qi_ridge_rmse, 3)

## Question 19 : Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?







In [ ]:
lasso_model = Lasso(alpha = 0.001)

In [ ]:
lasso_model.fit(x_train, y_train)

Lasso(alpha=0.001)

In [ ]:
lasso_model_weights = get_weights_df(lasso_model, x_train, "Lasso_Model_Weight")
lasso_model_weights

,Features,Lasso_Model_Weight
0,RH_weather_station,-0.049557
1,RH_in_teenager_room2,-0.000110
2,temp_in_kitchen,0.000000
3,Tdewpoint_weather_station,0.000000
4,visibility_weather_station,0.000000
5,pressure_waether_station,-0.000000
6,temp_weather_station,0.000000
7,RH_in_parents_room2,-0.000000
8,temp_in_parents_room,-0.000000
9,temp_in_teenager_room2,0.000000


In [ ]:
def count_non_zero_values(items) :
    counter = len([item for item in items if item != 0])
    return counter

In [ ]:
print(f'The number of features that have non_zero feature weights is {count_non_zero_values(lasso_model_weights["Lasso_Model_Weight"])}')

The number of features that have non_zero feature weights is 4


## Question 20 : What is the new RMSE with the Lasso Regression (in 3 decimal places)?



In [ ]:
lasso_model.fit(qi_x, qi_y)

Lasso(alpha=0.001)

In [ ]:
lasso_predict = lasso_model.predict(np.array(x_test["temp_in_living_room"]).reshape(-1, 1))

In [ ]:
qi_lasso_rmse = np.sqrt(mean_squared_error(np.array(x_test["temp_outside_building_northside"]), lasso_predict))
round(qi_lasso_rmse, 3) 

0.106